In [72]:
import pandas as pd
from os import listdir
import uuid
from datetime import date
import datetime
import numpy as np
import re

In [11]:
import warnings
warnings.filterwarnings("ignore")

## Functions

In [12]:
def assign_date_partnership(organizacion):
    dict_poynters = {'africa_check':datetime.date(2019, 8, 15), 
                    'la_silla_vacia':datetime.date(2020, 1, 31),
                     'chequeado':datetime.date(2018, 5, 1),
                     'animalpolitico':datetime.date(2018, 3, 1),
                     'colombiacheck': datetime.date(2020, 9, 28),
                     'ecuador_chequea':datetime.date(2020, 4, 1),
                     'larepublica':datetime.date(2020, 6, 1),
                    }
    
    if organizacion in dict_poynters.keys():
        return(dict_poynters[organizacion])
    else:
        return(np.nan)

def poynter_any_time(organizacion):
    dict_poynters = {'africa_check':datetime.date(2019, 8, 15), 
                    'la_silla_vacia':datetime.date(2020, 1, 31),
                     'chequeado':datetime.date(2018, 5, 1),
                     'animalpolitico':datetime.date(2018, 3, 1),
                     'colombiacheck': datetime.date(2020, 9, 28),
                     'ecuador_chequea':datetime.date(2020, 4, 1),
                     'larepublica':datetime.date(2020, 6, 1),
                    }
    
    if organizacion in dict_poynters.keys():
        return(1)
    else:
        return(0)

In [64]:
def include_missing_data():
    newspapers = pd.read_excel('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Bolivia_Project/data/13-fact-checks_virality/00-raw/missing_factchecks_links_newspapers.xlsx')
    social = pd.read_excel('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Bolivia_Project/data/13-fact-checks_virality/00-raw/missing_factchecks_links_social.xlsx')
    data = pd.concat([newspapers, social])
    data = data[~data['link_chequeo'].isna()]
    data['id_chequeo'] = data.apply(lambda x: str(uuid.uuid3(uuid.NAMESPACE_URL,x['link_chequeo'])), axis = 1)
    return(data)

In [86]:
def load_factchecks_newspapers():
    path = '/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Bolivia_Project/data/04-fakenews-Repository/04-fake_news_dataset/newspapers/Fact-Checkers/'
    data = []
    for i in listdir(path):
        try:
            data.append(pd.read_excel(path + i))
        except:
            pass
        
    factchecks = pd.concat(data)
    extra = include_missing_data()
    
    ## include missing factchecks
    factchecks = factchecks[~factchecks['id_desinformacion'].isin(list(extra.id_desinformacion))]
    factchecks = pd.concat([factchecks, extra]).drop(['Unnamed: 0'], axis = 1)
    factchecks = factchecks[['id_chequeo', 'organizacion', 'pais', 'titulo_chequeo', 'fecha_chequeo',
       'link_chequeo', 'id_desinformacion']]
    factchecks['facebook_partnership_date'] = factchecks.apply(lambda x: assign_date_partnership(x['organizacion']), axis = 1)
    factchecks['facebook_partnership'] = factchecks.apply(lambda x: poynter_any_time(x['organizacion']), axis = 1)    
    return(factchecks)


def load_factchecks_social():
    path = '/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Bolivia_Project/data/04-fakenews-Repository/04-fake_news_dataset/social/Fact-Checkers/'
    data = []
    for i in listdir(path):
        try:
            data.append(pd.read_excel(path + i))
        except:
            pass
        
    factchecks = pd.concat(data)
    extra = include_missing_data()
    
    ## include missing factchecks
    factchecks = factchecks[~factchecks['id_desinformacion'].isin(list(extra.id_desinformacion))]
    factchecks = pd.concat([factchecks, extra]).drop(['Unnamed: 0'], axis = 1)
    factchecks = factchecks[['id_chequeo', 'organizacion', 'pais', 'titulo_chequeo', 'fecha_chequeo',
       'link_chequeo', 'id_desinformacion']]
    factchecks['facebook_partnership_date'] = factchecks.apply(lambda x: assign_date_partnership(x['organizacion']), axis = 1)
    factchecks['facebook_partnership'] = factchecks.apply(lambda x: poynter_any_time(x['organizacion']), axis = 1)    
    return(factchecks)


def load_factchecks():
    social = load_factchecks_social()
    newspapers = load_factchecks_newspapers()
    africa = include_africa_check()[list(newspapers.columns)]
    
    all_data = pd.concat([social, newspapers, africa])
    all_data = all_data.drop_duplicates()
    all_data = all_data.rename(columns = {'id_chequeo':'id_factcheck'})
    return(all_data)


In [80]:
def include_africa_check():
    africa_check = pd.read_excel('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Bolivia_Project/data/04-fakenews-Repository/06-virality/africa/fact_checks/fact_checks_clean.xlsx')
    africa_check['pais'] = 'africa'
    africa_check['organizacion'] = 'africa_check'
    africa_check['titulo_chequeo'] = africa_check.apply(lambda x: obtain_title_chequeo(x['link_chequeo']), axis = 1)
    africa_check['facebook_partnership_date'] = africa_check.apply(lambda x: assign_date_partnership(x['organizacion']), axis = 1)
    africa_check['facebook_partnership'] = africa_check.apply(lambda x: poynter_any_time(x['organizacion']), axis = 1)    
    
    africa_check = africa_check.rename(columns = {'id_factcheck':'id_chequeo', 
                                                 'date_factcheck':'fecha_chequeo'})
    return(africa_check)

In [74]:
def obtain_title_chequeo(link):
    link = re.sub('[/]$', '', link).split('/')[-1:][0]
    link = re.sub('-', ' ', link)
    return(link)

## Apply functions

In [87]:
all_data = load_factchecks()

In [90]:
## misinformations with more than one factcheck
all_data[['id_desinformacion', 'id_factcheck']].dropna().groupby(['id_desinformacion']).count()[all_data[['id_desinformacion', 'id_factcheck']].dropna().groupby(['id_desinformacion']).count()['id_factcheck'] > 1]

,id_factcheck
id_desinformacion,
06da8c6f-e46b-3184-806e-aee131a7b92f,2
23070c8c-585e-3f92-9661-cf05227fd0ef,5
49eddb62-1f94-34c7-a79f-67f5a3a430ab,3
5915f43f-ab26-388e-b5f4-4e40b1958a8e,2
847949a6-6e6b-3d74-903f-4d903754751a,2
ac9c8595-5651-33b1-aacc-0404e24ccfe9,2
b13854c6-b0e9-3a63-aad5-891a91f7b8c7,2
bf35534c-56e7-3776-b705-05abb6cbe218,2
d8d37e06-c0ad-3341-a333-63d08f6f8ab1,2


## Save Data

In [91]:
all_data.to_excel('/Users/cblanesg/misinformation_socialmedia/data/1-factechecks/0-factchecks/factchecks.xlsx')